In [35]:
# Program to finder the best match of an unknown spectrum 
# in the folder "unknown", among the known spectrums wchich came from the
# folder "materials"

import glob
import os
import csv
import pickle


import numpy as np
import pylab
from __future__ import division



# Here should ne writen the satellite's responses we have
name_responses=['re', 's2A', 'wv3'];
num_resp=len(name_responses);

#retrieve of the unknown coefficients into the string coef_unk
unk = glob.glob('unknown/*');
num_unk=len(unk);
coef_unk=range(num_resp);
coef_unk[:]=[[] for i in coef_unk];

i_prove=0; # for proving-experiments

for x in unk:
    coef_temp=[];
    resp=open(x,'rU');
    csv_unk=csv.reader(resp);
    name_unk=os.path.basename(x).replace('.csv','').replace('.CSV','');
    
    i_resp=0;
    for xy in name_responses:
        if xy in name_unk:
            # this is a clumsy step, but so far necesarry to work with csv
            for xyz in csv_unk:
                for i in xyz:
                    coef_temp.append(float(i));
            coef_temp.append(name_unk);
            coef_unk[i_resp].append(coef_temp);
        i_resp+=1;
    resp.close();
    
# Retrieve the coefficients of the materials we know
mat=glob.glob('coefficients/*');
# Retrieve the values and the names od the known coefficients
coef_mat=range(num_resp);
coef_mat[:]=[[] for i in coef_mat];
name_mat=range(num_resp);
name_mat[:]=[[] for i in name_mat];

for x in mat:
    mat_temp=open(x,'rU');
    i_resp=0;
    for xy in name_responses:
        if xy in os.path.basename(x):
            if 'coef' in os.path.basename(x):
                coef_mat[i_resp]=pickle.load(mat_temp);
            elif 'mat' in os.path.basename(x):
                name_mat[i_resp]=pickle.load(mat_temp);
        i_resp+=1;
    mat_temp.close();
    
# Now, make the matching program
i_resp=0;
# next x to swamp the unknown files
for x in name_responses:
    # next i to swamp the unknown coefficients for i_resp sat
    for i in range(len(coef_unk[i_resp])):
        #unlikely the smallest value smaller than this
        metro_min=1000000; 
        # j counts the known coeff for i_resp sat
        for j in range(len(coef_mat[i_resp])):
            coef_mat_temp=coef_mat[i_resp];
            metro=0;
            #Here it is made the measurement
            for k in range(len(coef_unk[i_resp][i])-1):
                metro+=(coef_unk[i_resp][i][k]-coef_mat_temp[j,k])**2;
            if metro<metro_min:
                metro_min=metro;
                j_min=j;

        print 'The most similar to',coef_unk[i_resp][i][k+1],'is'+name_mat[i_resp][j_min],'in',x;
    i_resp+=1;


The most similar to changed_1.1.s2A is1.1 in s2A
The most similar to changed_2.1.s2A is2.1 in s2A
The most similar to changed_2.3.s2A is2.3 in s2A
The most similar to changed_3.2.s2A is3.2 in s2A
The most similar to changed_4.4.s2A is4.4 in s2A
The most similar to changed_3.2.wv3 is3.2 in wv3
